## 01. 라이브러리 로드

In [231]:
import pandas as pd
import numpy as np

## 02. 데이터 로드
- yearvolume : 티커, 연간변동성, 회계년도
- cagr : 거래소코드, 회계년도, CAGR  
- benchmark : 회사명, 거래소코드, 회계년도, 수정종가, 초과수익률, 수익률(%)
- beta : 티커, 베타, 회계년도
- sharpe : 티커, 연간수익률, 연간변동성, 샤프지수, 회계년도
- original : 전처리 후 데이터셋
- target : 회사명, 회계년도, target_pred

In [232]:
yearvolume = pd.read_excel('data/krx/연간변동성.xlsx')
cagr = pd.read_excel('data/krx/누적_CAGR.xlsx')
benchmark = pd.read_excel('data/krx/초과수익률.xlsx')
beta = pd.read_excel('data/krx/beta.xlsx')
sharpe = pd.read_excel('data/krx/sharpe.xlsx')
original = pd.read_excel('data/전처리후데이터셋.xlsx')
target = pd.read_excel('data/predict.xlsx')

## 03. 데이터 병결합을 위한 컬럼명 재정의 및 데이터 결합
- original : 회사명, 거래소코드, 회계년도 컬럼 추출 및 재정의
- target : 회사명, 회계년도, target_class 컬럼 추출

In [233]:
original = original[['회사명', '거래소코드', '회계년도']]
original['회계년도'] = original['회계년도'].astype(str).str[:4].astype(int)
target = target[['회사명', '회계년도', 'target_pred']]

#### original 기준으로 left 조인

In [234]:
df = pd.merge(original, target, on=['회사명', '회계년도'], how='left')

#### 회계년도 기준 2019년 이후로 추출 후 target 변수 결측치 0으로 채우기

In [235]:
df = df[df['회계년도'] > 2019]
df['target_pred'].fillna(0, inplace=True)

C:\Users\tmddn\AppData\Local\Temp\ipykernel_13972\829441471.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['target_pred'].fillna(0, inplace=True)


#### merge_1
- df와 benchmark 결합 : df 기준으로 left 조인 
- 거래소코드 : str 변환
- 회사명, 수정종가 컬럼 제거

In [236]:
df['거래소코드'] = df['거래소코드'].astype(str).str.zfill(6)
benchmark['거래소코드'] = benchmark['거래소코드'].astype(str).str.zfill(6)
benchmark.drop(columns=['회사명', '수정종가'], inplace=True)
merge_1 = pd.merge(df, benchmark, on=['거래소코드', '회계년도'], how='left')

#### merge_2
- merge_1 과 yearvolumn 결합 : merge_1 기준으로 left 조인
- 거래소코드 : 컬럼명 재정의

In [237]:
yearvolume = yearvolume.rename(columns={'티커': '거래소코드'})
merge_2 = pd.merge(merge_1, yearvolume, on=['거래소코드', '회계년도'], how='left')

#### merge_3
- merge_2 와 beta 결합 : merge_2 기준으로 left 조인
- 거래소코드 : 컬럼명 재정의

In [238]:
beta = beta.rename(columns={'티커': '거래소코드'})
merge_3 = pd.merge(merge_2, beta, on=['거래소코드', '회계년도'], how='left')

#### merge_4
- merge_3과 sharpe 결합 : merge_3 기준으로 left 조인
- 거래소코드 : 컬럼명 재정의
- 샤프지수 : 무한대 값 -> 널 값으로 처리
- 연간수익률, 연간변동성 컬럼 제거

In [239]:
sharpe = sharpe.rename(columns={'티커': '거래소코드'})
sharpe.drop(columns=['연간수익률', '연간변동성'], inplace=True)
sharpe.replace([np.inf, -np.inf], np.nan, inplace=True)
merge_4 = pd.merge(merge_3, sharpe, on=['거래소코드', '회계년도'], how='left')

#### merge_5
- merge_4와 cagr 결합 : merge_4 기준으로 left 조인

In [240]:
merge_5 = pd.merge(merge_4, cagr, on=['거래소코드', '회계년도'], how='left')

#### 결측치 제거

In [241]:
merge_5.dropna(inplace=True)

## 04. 데이터 저장
- 중복행 제거 후 저장

In [245]:
key_cols = ['회사명', '거래소코드', '회계년도']

merge_5 = merge_5.drop_duplicates(subset=key_cols)

In [246]:
merge_5.to_excel('data/test_predict.xlsx', index=False)